In [95]:
import sys
import collections
import json
import csv
import os
import time
import subprocess
from datetime import datetime

In [96]:
#Parsing Printout LOG
def percent_hist(percent, hist):
    if not hist:
        return 0
    key_lst = hist.keys()
    key_lst = sorted(key_lst, key=lambda t:t[0])
    num = 0.0
    for k,v in hist.items():
        num += v
    pos = percent*num
    left_num = 0
    count = 0
    for k in key_lst:
        v = hist[k]
        count +=1
        if (left_num <= pos and left_num +v > pos) or count == len(hist.keys()):
            range = k[1]-k[0]
            ratio = (pos-left_num) / v
            return  k[0] + range*ratio
        else:
            left_num += v

def avg_lst(lst):
    if (len(lst) == 0):
        return 0.0
    return sum(lst) / len(lst)

def avg_dif_lst(lst):
    total = 0.0
    for i in range(1, len(lst)):
        if lst[i]  == ',' or lst[i-1] == ',':
            continue
        if lst[i] >= lst[i-1]:
            total+=(float(lst[i])-float(lst[i-1]))
    return total 

def parse_rocksdb_log(output_file):
    comp_cnt_lst = []
    comp_duration_lst = []
    comp_duration_b_lst = []
    comp_rd_lst = []
    comp_wr_lst = []
    comp_score_lst = []
    filename = output_file
    
    with open(filename, 'r') as f:
        iops = 0
        wr_avg = 0
        wr_p50 = 0
        wr_p99 = 0
        wr_p999 = 0
        wr_p9999= 0
        rd_avg = 0
        rd_p50=0
        rd_p99=0
        rd_p999=0
        rd_p9999=0
        total_iops = 0
        hit_rate=[]
        miss_rate=[]
        rd_hist = collections.OrderedDict()
        wr_hist = collections.OrderedDict()
        comp_cnt = collections.OrderedDict()

        line_lst = f.readlines()
        find_end = False
        after_fillseq = False
        find_cache_hit = False
        i = 0
        while(i < len(line_lst)):
            if line_lst[i].find('ycsb') != -1:
                value_lst = line_lst[i].split()
                iops = value_lst[4]
                total_iops += int(iops)
                find_end = True
                i += 1
            elif find_end and line_lst[i].find('Microseconds per write') != -1:
                value_lst = line_lst[i + 1].split()
                wr_avg = float(value_lst[3])
                i+=5
                while(True):
                    if(line_lst[i] == '\n'):
                        break
                    value_lst = line_lst[i].split()
                    left_point = float(value_lst[1].replace(',',''))
                    right_point = float(value_lst[2])
                    if (left_point, right_point) not in wr_hist.keys():
                        wr_hist[(left_point, right_point)] = 0.0
                    wr_hist[(left_point, right_point)] += float(value_lst[4])
                    i += 1
            elif find_end and line_lst[i].find('Microseconds per read') != -1:
                value_lst = line_lst[i + 1].split()
                rd_avg += float(value_lst[3])
                i+=5
                while(True):
                    if(line_lst[i] == '\n'):
                        break
                    value_lst = line_lst[i].split()
                    left_point = float(value_lst[1].replace(',',''))
                    right_point = float(value_lst[2])
                    if (left_point, right_point) not in rd_hist.keys():
                        rd_hist[(left_point, right_point)] = 0.0
                        rd_hist[(left_point, right_point)] += float(value_lst[4])
                    i += 1
            elif line_lst[i].find('fillseq') != -1:
                after_fillseq = True
                i += 1
            elif  line_lst[i].find('** Compaction Stats [default] **') != -1:
                for lev in range(3):
                    value_lst = line_lst[i + 3 + lev].split()
                    if(value_lst[0].find('Low')!=-1):
                    #TODO: LOW/HIGH Compact. Stat.
                        break
                    if value_lst[0] not in comp_cnt.keys():
                        comp_cnt[value_lst[0]] = []
                        comp_cnt[value_lst[0]].append(value_lst[-6])

                i += 1
            elif line_lst[i].find('rocksdb.block.cache.data.hit') != -1:
                find_cache_hit = True
                value_lst = line_lst[i].split()
                hit_rate.append(float(value_lst[3]))
                i +=1
            elif line_lst[i].find('rocksdb.block.cache.data.miss') != -1:
                value_lst = line_lst[i].split()
                miss_rate.append(float(value_lst[3]))
                i+=1
            else:
                i+=1
        for k in comp_cnt.keys():
            comp_cnt[k].append(',')
            
       
        wr_p50 = percent_hist(0.5, wr_hist)
        wr_p99 = percent_hist(0.99, wr_hist)
        wr_p999 = percent_hist(0.999, wr_hist)
        wr_p9999 = percent_hist(0.9999, wr_hist)
     
        rd_p50 = percent_hist(0.5, rd_hist)
        rd_p99 = percent_hist(0.99, rd_hist)
        rd_p999 = percent_hist(0.999, rd_hist)
        rd_p9999 = percent_hist(0.9999, rd_hist)
        L_comp_cnt = []
        for k in ['L0', 'L1', 'L2', 'L3']:
            if k in comp_cnt.keys():
                L_comp_cnt.append(avg_dif_lst(comp_cnt[k]))
            else:
                L_comp_cnt.append(0.0)
        if(find_cache_hit):
            cache_hit =  avg_lst(hit_rate) / (avg_lst(hit_rate) + avg_lst(miss_rate))
        else:
            cache_hit = 0
        print("total_iops,", total_iops)
        print("cache_hit,", cache_hit)
        print("L_comp_cnt[0],", L_comp_cnt[0])
        print("L_comp_cnt[1],", L_comp_cnt[1])
        print("L_comp_cnt[2],", L_comp_cnt[2])
        print("L_comp_cnt[3],", L_comp_cnt[3])
        print("wr_avg,", wr_avg)
        print("wr_p50,", wr_p50)
        print("wr_p99,", wr_p99)
        print("wr_p999,", wr_p999)
        print("wr_p9999,", wr_p9999)
        print("rd_avg,", rd_avg)
        print("rd_p50,", rd_p50)
        print("rd_p99,", rd_p99)
        print("rd_p999,", rd_p999)
        print("rd_p9999,", rd_p9999)

In [97]:
#Run sudo without passwd
def rocksdb(flagfile, output_file):
    open(output_file, 'w').close()
    cmd = "sudo ../db_bench --flagfile={} > {}".format(flagfile, output_file)
    x = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
    parse_rocksdb_log(output_file)

In [106]:
my_rocksdb_para_ = {
        '--disable_seek_compaction' : 1,
        '--mmap_read' : 0,
        '--statistics' : 1,
        '--histogram' : 1,
        '--key_size' : 16,
        '--value_size': 1024,
        '--block_size' : 4096,
        '--bloom_bits' : 10,
        '--cache_numshardbits' : 4,
        '--open_files' : 500000,
        '--db' : '/rocksdb',
        '--sync' : 0,
        '--compression_type' : 'none',
        '--stats_interval' : 1000000,
        '--compression_ratio' : 1,
        '--target_file_size_base' : 67108864,
        '--target_file_size_multiplier' : 2,
        '--write_buffer_size' : 67108864,
        '--max_write_buffer_number' : 6,
        '--min_write_buffer_number_to_merge' : 2,
        '--num_levels' : 10,
        '--delete_obsolete_files_period_micros' : 3000000000,
        '--stats_per_interval' : 1,
        '--max_bytes_for_level_base' : 671088640,
        '--level0_slowdown_writes_trigger' : 20,
        '--level0_stop_writes_trigger' : 24,
        '--threads' : 6,
        '--duration' : 20,
        '--disable_wal' : 1,
        '--use_existing_db' : 0,
        '--num' : 1000000,
        '--op_num' : 500000,
        '--nvmf_target' : '192.168.1.11',
        '--cache_size' : 4096000000,   
        '--hotspot_num' : 1,
        '--max_background_flushes' : 1,
        '--max_background_compactions' : 4,
        '--max_bytes_for_level_multiplier' : 4,
        '--benchmarks' : 'fillseq,ycsbwkldf[-X1]',
        '--skewness' : 0.9,
        '--level0_file_num_compaction_trigger' : 8,
        '--ycsba_read_ratio' : 50,
        '--use_direct_io_for_flush_and_compaction' : 1,
        '--fs_uri' : 'zenfs://dev:nullb0'
    }

def gen_rocksdb_flagfile(flag_file_name, rocksdb_para):
    open(flag_file_name, 'w').close()
    for k, v in rocksdb_para.items():
        os.system('echo {}={} >> {}'.format(k, v, flag_file_name))
        
        
gen_rocksdb_flagfile('./ycsba_flags.txt', my_rocksdb_para_)

In [ ]:
#Run sudo without passwd
os.system('sudo ./nullblk-zoned.sh 4096 64 128 128')
os.system('sudo rm -rf /home/cxyzhao/znslog/*')
os.system('sudo ../plugin/zenfs/util/zenfs mkfs --zbd=nullb0 --aux_path=/home/cxyzhao/znslog')
rocksdb('./ycsba_flags.txt', './out')
os.system('sudo ./delete_nullblk.sh 0')